In [1]:
import json
import requests
import pandas as pd
import os

In [2]:
r = requests.get("https://pan-unit42.github.io/playbook_viewer/playbooks.json")

index = []
if r.status_code == 200:
    print("Loading playbooks")
    
    pb_list = r.json()
    
    for report in pb_list:
        #print(f"Title: {report['title']} File Name: {report['pb_file']}")
        stix_name = report['pb_file']
        
        #search pan unit 42 block
        s = requests.get(f"https://www.paloaltonetworks.com/search/unit42search#q={report['title']}")
        
        if .status_code == 200:
            print(
        index.append({"Title":report['title'],"File Name":stix_name})
        
        if os.path.isfile(stix_name): continue
        
        with open(report['pb_file'],'w') as file:
            headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
            rf = requests.get(f"https://pan-unit42.github.io/playbook_viewer/playbook_json/{stix_name}",headers=headers)
             
            if r.status_code == 200:
                json.dump(rf.json(),file)
            else:
                print(r.status_code)
                
    index_df = pd.DataFrame(index)
    
    print("Done!")

Loading playbooks
Done!


In [3]:
index_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83 entries, 0 to 82
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Title      83 non-null     object
 1   File Name  83 non-null     object
dtypes: object(2)
memory usage: 1.4+ KB


In [4]:
index_df.head(4)

,Title,File Name
0,EVASIVE SERPENS,evasive-serpens.json
1,FIGHTING URSA,fighting-ursa.json
2,PRYING LIBRA,pryinglibra.json
3,OBSCURE SERPENS,obscureserpens.json


In [5]:
index_df.tail(4)

,Title,File Name
79,TRIGONA-RANSOMWARE,trigona-ransomware.json
80,JAVASCRIPT-INJECTION,javascript-injection.json
81,CRYPTOCLIPPY,cryptoclippy.json
82,MUDDLED LIBRA,muddled-libra.json


In [6]:
index_df.to_csv("pan_unit42.csv",index=False)